<a href="https://colab.research.google.com/github/kavyakalivarapu-lab/kavya.github.io/blob/main/labelling_modelling_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers datasets  scikit-learn tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 9.3 MB/s eta 0:00:00


In [3]:
!pip uninstall -y torch torchvision torchaudio

In [4]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

Looking in indexes: https://download.pytorch.org/whl/cpu
  Using cached https://download.pytorch.org/whl/cpu/torch-2.6.0%2Bcpu-cp311-cp311-linux_x86_64.whl.metadata (26 kB)
  Using cached https://download.pytorch.org/whl/cpu/torchvision-0.21.0%2Bcpu-cp311-cp311-linux_x86_64.whl.metadata (6.1 kB)
  Using cached https://download.pytorch.org/whl/cpu/torchaudio-2.6.0%2Bcpu-cp311-cp311-linux_x86_64.whl.metadata (6.6 kB)
Using cached https://download.pytorch.org/whl/cpu/torch-2.6.0%2Bcpu-cp311-cp311-linux_x86_64.whl (178.7 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 35.9 MB/s eta 0:00:00


In [6]:
import sys
print(sys.version)


3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0]


In [9]:
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import pandas as pd
from tqdm import tqdm  # Progress bar

In [6]:
# Set CPU threading for better performance
torch.set_num_threads(8)

In [10]:
# Load model and tokenizer correctly
model_path = "unitary/unbiased-toxic-roberta"
tokenizer = RobertaTokenizer.from_pretrained(model_path)
model = RobertaForSequenceClassification.from_pretrained(model_path)

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [11]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

In [13]:
# Load dataset
file_path = "/content/drive/My Drive/TW/Dataset/collection/cleaned_translated_dataset.csv"  # Update path if needed
df = pd.read_csv(file_path)

In [14]:
# Define batch size for CPU processing
BATCH_SIZE = 16

In [15]:
# Function to classify a batch of texts
def classify_batch(texts):
    inputs = tokenizer(texts, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=1).tolist()
    return ["bully" if pred == 1 else "not bully" for pred in predictions]

In [16]:
# Process data in batches
labels = []
for i in tqdm(range(0, len(df), BATCH_SIZE), desc="Labeling Progress"):
    batch_texts = df["Translated_Text"].iloc[i:i+BATCH_SIZE].tolist()
    batch_labels = classify_batch(batch_texts)
    labels.extend(batch_labels)

Labeling Progress: 100%|██████████| 622/622 [32:41<00:00,  3.15s/it]


In [17]:
# Add labels to DataFrame
df["Label"] = labels




In [19]:
# Save labeled dataset
df.to_csv("/content/drive/My Drive/TW/Dataset/collection/bert_labeled_dataset.csv", index=False)

print("✅ BERT Labeling Completed! Saved as 'bert_labeled_dataset.csv'.")

✅ BERT Labeling Completed! Saved as 'bert_labeled_dataset.csv'.


**Fine-Tuning BERT for Cyberbullying Detection**

In [ ]:
import torch
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from torch.nn import functional as F

In [ ]:
# Set device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Load labeled dataset
file_path = "bert_labeled_dataset.csv"
df = pd.read_csv(file_path)

In [ ]:
# Encode labels (convert "bully" → 1, "not bully" → 0)
df["Label"] = df["Label"].map({"bully": 1, "not bully": 0})

In [ ]:
# Load tokenizer
model_name = "bert-base-uncased"  # You can also use "bertweet-base"
tokenizer = BertTokenizer.from_pretrained(model_name)

In [ ]:
# Tokenization function
class CyberbullyingDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = self.texts[index]
        label = self.labels[index]
        encoding = self.tokenizer(text, padding="max_length", truncation=True, max_length=self.max_len, return_tensors="pt")

        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "label": torch.tensor(label, dtype=torch.long),
        }


In [ ]:
# Split data into train and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(df["Translated_Text"], df["Label"], test_size=0.2, random_state=42)


In [ ]:
# Create dataset objects
train_dataset = CyberbullyingDataset(train_texts.tolist(), train_labels.tolist(), tokenizer)
val_dataset = CyberbullyingDataset(val_texts.tolist(), val_labels.tolist(), tokenizer)

In [ ]:
# Create DataLoader
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# Load pre-trained BERT model
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)
model.to(device)

In [ ]:
# Optimizer & Loss Function
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
# Training Loop
num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in train_loader:
        input_ids, attention_mask, labels = batch["input_ids"].to(device), batch["attention_mask"].to(device), batch["label"].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask)
        loss = criterion(outputs.logits, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}, Training Loss: {avg_train_loss:.4f}")


In [ ]:
# Save the fine-tuned model
model.save_pretrained("fine_tuned_bert_model")
tokenizer.save_pretrained("fine_tuned_bert_model")

print("Fine-tuning completed! Model saved as 'fine_tuned_bert_model'.")

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# ---------------- EVALUATION ---------------- #

def evaluate_model(model, val_loader):
    model.eval()
    predictions, true_labels = [], []

    with torch.no_grad():
        for batch in val_loader:
            input_ids, attention_mask, labels = batch["input_ids"].to(device), batch["attention_mask"].to(device), batch["label"].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1).cpu().numpy()
            labels = labels.cpu().numpy()

            predictions.extend(preds)
            true_labels.extend(labels)

    accuracy = accuracy_score(true_labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predictions, average="binary")
    conf_matrix = confusion_matrix(true_labels, predictions)

    print("\n--- Model Evaluation ---")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-Score: {f1:.4f}")

    # Plot Confusion Matrix
    plt.figure(figsize=(5, 5))
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=["Not Bully", "Bully"], yticklabels=["Not Bully", "Bully"])
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title("Confusion Matrix")
    plt.show()




In [ ]:
# Call evaluation function
evaluate_model(model, val_loader)